# <center>Generating Music Using GAN</center>

This workbook will implement modified code from [this](https://github.com/corynguyen19/midi-lstm-gan) GitHub repo.


In [1]:
from __future__ import print_function, division
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
import time
import os
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

Using TensorFlow backend.


## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [2]:
def get_notes(path):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                song.append('.'.join(str(n) for n in element.normalOrder))
        notes.append(song)

    return notes

def get_notes_with_key(path, filter_key, mode):
    """
        Gets all notes and chords from midi files where the key matches the string input

        Parameters
        ----------
        path : str
            The path to the file
        filter_key : str
            The string to filter the key on
        mode : int
            The type of key used where:
                0 - key
                1 - major/minor
                else - key and major/minor
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        # Only use music of the same key
        key = midi.analyze('key')
        if(mode==0):
            key_string = str(key.tonic.name)
        elif(mode==1):
            key_string = str(key.mode)
        else:
            key_string = str(key.tonic.name + key.mode)
            
        if(key_string==filter_key):
            print("Parsing %s" % file)
            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    song.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    song.append('.'.join(str(n) for n in element.normalOrder))
            notes.append(song)

    return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 0
input_dir_names = ["test", "Pokemon", "LoZ", "Pokemon GSC", "ABBA"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
# example of each mode: 0 - C, 1 - major, 2 - Cmajor
# notes = get_notes_with_key(input_path, "minor", 1)
notes = get_notes(input_path)

Parsing ../test MIDIs\Undertale - ASGORE.mid
Parsing ../test MIDIs\Undertale - Battle Against a True Hero.mid


I will now use an algo to determine the key of each song

In [3]:
def print_key(path):
    key_count = dict()
    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        key = midi.analyze('key')
        key_string = key.tonic.name + key.mode
        if (key_string in key_count): 
            key_count[key_string] += 1
        else: 
            key_count[key_string] = 1
        print(key.tonic.name, key.mode)
    return key_count

input_dir_choice = 1
input_dir_names = ["Pokemon", "LoZ", "Pokemon GSC", "ABBA"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
key_count = print_key(input_path)
key_count

Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - Cinnabar Island (HGSS Version).mid
G major
Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - S.S. Aqua .mid
G major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
C# major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bicycle.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bug Catching Contest.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Burned Tower.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Champion Battle.mid
G# minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Cherrygrove CityMahogany Town.mid
F major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dance Theatre.mid
A minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dark Cave.mid
A- major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dragons Den.mid
C# minor
Parsing ../Pokemon GSC MIDIs\Pokemon

{'Gmajor': 5,
 'C#major': 1,
 'Eminor': 3,
 'G#minor': 3,
 'Fmajor': 3,
 'Aminor': 3,
 'A-major': 4,
 'C#minor': 3,
 'Cmajor': 8,
 'Emajor': 4,
 'Dmajor': 8,
 'B-minor': 4,
 'E-major': 3,
 'Bmajor': 2,
 'Amajor': 2,
 'Fminor': 1,
 'Cminor': 1,
 'Dminor': 1,
 'F#major': 1,
 'E-minor': 1}

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [3]:
possibleNotes = set([item for sublist in notes for item in sublist])

n_vocab = len(possibleNotes)
possibleNotes, n_vocab

({'0',
  '0.3',
  '0.4',
  '0.4.7',
  '0.5',
  '1',
  '1.5',
  '1.5.8',
  '1.7',
  '10',
  '10.3',
  '11.2',
  '2',
  '2.4',
  '2.4.9',
  '2.5',
  '2.5.9',
  '2.7',
  '3',
  '3.4',
  '3.7',
  '3.7.10',
  '3.8',
  '4',
  '4.5.10',
  '4.7',
  '4.9',
  '5',
  '5.10',
  '5.7',
  '5.7.0',
  '5.8',
  '5.8.0',
  '5.9',
  '5.9.0',
  '7',
  '7.0',
  '7.10',
  '7.11',
  '7.9',
  '8',
  '8.0',
  '9.0',
  '9.2',
  'A1',
  'A2',
  'A3',
  'A4',
  'A5',
  'B-1',
  'B-2',
  'B-3',
  'B-4',
  'B-5',
  'B2',
  'B5',
  'C#2',
  'C#3',
  'C#5',
  'C#6',
  'C#7',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D7',
  'E-2',
  'E-3',
  'E-4',
  'E-5',
  'E-6',
  'E-7',
  'E2',
  'E3',
  'E4',
  'E5',
  'F2',
  'F3',
  'F4',
  'F5',
  'F6',
  'G#1',
  'G#2',
  'G#3',
  'G#4',
  'G#5',
  'G#6',
  'G1',
  'G2',
  'G3',
  'G4',
  'G5',
  'G6'},
 99)

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [4]:
def prepare_sequences(notes, allNotes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    step_size = 1

    # get all pitch names
    pitchnames = sorted(allNotes)

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])

    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input between 0 and 1
    network_input = network_input / float(n_vocab)
    
    # normalize input between -1 and 1
#     network_input = (network_input / float(n_vocab))*2 - 1

    # Make one-hot-encoding
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

network_input, network_output = prepare_sequences(notes, possibleNotes, n_vocab)
len(network_input)

3620

## Constructing the model


In [5]:
class GAN():
    def __init__(self, rows, network_input):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator(network_input)
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        self.outputDest = '../output/GAN_' + str(int(time.time())) + '/'
        if not os.path.exists(self.outputDest):
            os.makedirs(self.outputDest)

    def build_discriminator(self, network_input):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
        model.add(Dropout(0.3))
        model.add(Bidirectional(CuDNNLSTM(512, return_sequences=True)))
        model.add(Dropout(0.3))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(256))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(128, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, X_train, y_train, epochs, batch_size=128, sample_interval=50):

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs+1):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            self.disc_loss.append(d_loss[0])
            self.gen_loss.append(g_loss)
            
              # Save model often
            if epoch % sample_interval == 0:
                self.generator.save(self.outputDest + "GANmodel_weights_" + str(epoch) + ".hdf5")
                
    def generate_notes(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(notes)
        n_vocab = len(pitchnames)
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        print(predictions)
        pred_notes = [(x+1)*n_vocab/2 for x in predictions[0]]
        pred_notes = [int_to_note[min(int(x), n_vocab - 1)] for x in pred_notes]
        print(pred_notes)
        
        return pred_notes
        
    def create_midi(self, prediction_output, filename):
        offset = 0
        output_notes = []

        # create note and chord objects based on the values generated by the model
        for pattern in prediction_output:
            # pattern is a chord
            if ('.' in pattern) or pattern.isdigit():
                notes_in_chord = pattern.split('.')
                notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            # pattern is a note
            else:
                new_note = note.Note(pattern)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)
            # increase offset each iteration so that notes do not stack
            offset += 0.5

        midi_stream = stream.Stream(output_notes)
        midi_stream.write('midi', fp='{}.mid'.format(filename))
        
    def generate_midi_from_saves(self, input_notes):
        # Have each model make a song
        count = 0
        for model_path in glob.glob(self.outputDest + "*.hdf5"):
            print("Composing from %s" % model_path)
            self.generator.load_weights(model_path)
            prediction_notes = self.generate_notes(input_notes)
            self.create_midi(prediction_notes, self.outputDest + 'GAN_output_' + str(count))
            print(self.outputDest + 'GAN_output_' + str(count))
            count += 1
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig(self.outputDest + 'GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [6]:
gan = GAN(100, network_input)
gan.train(X_train=network_input, y_train=network_output, epochs=2000, batch_size=20, sample_interval=100)
gan.plot_loss()

W1009 13:14:28.511108 14132 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1009 13:14:28.511108 14132 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1009 13:14:30.134441 14132 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1009 13:14:30.396397 14132 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6299648   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(20, 1000), b.shape=(1000, 128), m=20, n=128, k=1000
	 [[{{node sequential_2/dense_3/MatMul}}]]
	 [[sequential_2/reshape_1/Reshape/_131]]
  (1) Internal: Blas GEMM launch failed : a.shape=(20, 1000), b.shape=(1000, 128), m=20, n=128, k=1000
	 [[{{node sequential_2/dense_3/MatMul}}]]
0 successful operations.
0 derived errors ignored.

In [122]:
gan.generate_midi_from_saves(possibleNotes)

Composing from ../output/GAN_1570439027\GANmodel_weights_0.hdf5
[[[-0.94293034]
  [ 0.9511678 ]
  [ 0.95636314]
  [-0.21992657]
  [-0.6503713 ]
  [ 0.99673396]
  [ 0.9872717 ]
  [ 0.7317666 ]
  [-0.6498992 ]
  [ 0.23074849]
  [-0.1567165 ]
  [ 0.89335775]
  [-0.731017  ]
  [ 0.9066702 ]
  [ 0.15424827]
  [ 0.9639665 ]
  [ 0.9858628 ]
  [ 0.974552  ]
  [ 0.6970632 ]
  [-0.44110644]
  [ 0.20285414]
  [-0.9963693 ]
  [ 0.10762044]
  [ 0.64688444]
  [-0.69539547]
  [ 0.31526315]
  [-0.8685319 ]
  [ 0.9949453 ]
  [-0.96725065]
  [ 0.10382608]
  [ 0.5636406 ]
  [-0.94493407]
  [ 0.9792849 ]
  [ 0.59865683]
  [ 0.13042538]
  [ 0.96497583]
  [-0.98510236]
  [ 0.616735  ]
  [ 0.962668  ]
  [ 0.92874897]
  [ 0.9901613 ]
  [-0.52822757]
  [-0.47610646]
  [-0.9229445 ]
  [ 0.23479456]
  [-0.99814975]
  [-0.9073559 ]
  [-0.9771247 ]
  [-0.47023472]
  [ 0.8450416 ]
  [-0.29251167]
  [-0.72006226]
  [ 0.44590446]
  [-0.8185839 ]
  [-0.96681917]
  [ 0.5594001 ]
  [ 0.9965545 ]
  [ 0.8174353 ]
  [ 0.64

../output/GAN_1570439027/GAN_output_4
Composing from ../output/GAN_1570439027\GANmodel_weights_1300.hdf5
[[[ 0.9959104 ]
  [ 0.9725371 ]
  [ 0.4277731 ]
  [ 0.6466447 ]
  [-0.2397645 ]
  [-0.31211254]
  [-0.4867928 ]
  [-0.99711585]
  [-0.3768475 ]
  [ 0.8301846 ]
  [-0.3029596 ]
  [ 0.5598421 ]
  [ 0.5407779 ]
  [-0.01950943]
  [-0.8009113 ]
  [ 0.91839206]
  [-0.6776985 ]
  [-0.8608296 ]
  [-0.2922929 ]
  [ 0.9313903 ]
  [-0.8628891 ]
  [-0.9652906 ]
  [-0.98599225]
  [-0.8582042 ]
  [-0.03185019]
  [-0.8096114 ]
  [-0.6344647 ]
  [-0.31025332]
  [ 0.8177382 ]
  [ 0.32978398]
  [-0.8172393 ]
  [-0.8706268 ]
  [-0.45500994]
  [ 0.19768581]
  [ 0.9029304 ]
  [ 0.59354913]
  [ 0.2643258 ]
  [-0.8443395 ]
  [ 0.05900037]
  [-0.83505774]
  [ 0.6667676 ]
  [-0.7955165 ]
  [-0.14817132]
  [-0.7497144 ]
  [ 0.9561483 ]
  [ 0.93773234]
  [ 0.9599365 ]
  [ 0.8961353 ]
  [ 0.8280164 ]
  [ 0.36768773]
  [-0.89846784]
  [ 0.96327704]
  [-0.5269395 ]
  [ 0.9641858 ]
  [ 0.9415521 ]
  [ 0.7080726 ]

[[[ 0.995284  ]
  [ 0.54836345]
  [ 0.99779606]
  [ 0.816773  ]
  [ 0.12112167]
  [ 0.9028896 ]
  [-0.836984  ]
  [-0.49362987]
  [-0.9104736 ]
  [-0.9746636 ]
  [-0.32649818]
  [ 0.8401226 ]
  [-0.7671018 ]
  [-0.926077  ]
  [ 0.8219258 ]
  [ 0.5970247 ]
  [-0.7947639 ]
  [-0.5498568 ]
  [ 0.45789656]
  [ 0.8540513 ]
  [ 0.38306278]
  [ 0.42673478]
  [ 0.02450535]
  [ 0.8834447 ]
  [-0.34574944]
  [-0.82318693]
  [-0.18124941]
  [ 0.36039484]
  [-0.72288764]
  [ 0.80476207]
  [ 0.90168434]
  [-0.89069605]
  [-0.9989071 ]
  [ 0.5268345 ]
  [ 0.25753742]
  [-0.6400441 ]
  [-0.5927043 ]
  [ 0.9612405 ]
  [ 0.06259491]
  [-0.45570564]
  [-0.74284804]
  [-0.24707569]
  [ 0.32256004]
  [ 0.7661398 ]
  [-0.01301926]
  [ 0.8558969 ]
  [-0.7850193 ]
  [ 0.96592945]
  [-0.95488614]
  [-0.9710496 ]
  [ 0.8272846 ]
  [-0.9614878 ]
  [ 0.2662847 ]
  [-0.8359278 ]
  [ 0.8097777 ]
  [-0.62775123]
  [ 0.55750465]
  [ 0.03078178]
  [ 0.21106218]
  [ 0.97799844]
  [-0.13217326]
  [-0.7452792 ]
  [ 0.94

[[[ 0.97451854]
  [-0.69874775]
  [ 0.9469765 ]
  [-0.6900997 ]
  [-0.7647221 ]
  [ 0.04223719]
  [-0.889413  ]
  [-0.8161018 ]
  [ 0.36718702]
  [-0.34000787]
  [-0.20417714]
  [-0.5971832 ]
  [-0.1212225 ]
  [ 0.8490779 ]
  [-0.6295926 ]
  [ 0.39084482]
  [ 0.04012639]
  [-0.55831397]
  [ 0.35601702]
  [ 0.9843732 ]
  [ 0.5023697 ]
  [-0.2765126 ]
  [ 0.3541942 ]
  [ 0.4019861 ]
  [-0.5220711 ]
  [ 0.74961567]
  [ 0.9826397 ]
  [-0.48996878]
  [ 0.6116426 ]
  [-0.00519587]
  [ 0.662876  ]
  [-0.38731268]
  [-0.12436269]
  [ 0.13401459]
  [-0.5486989 ]
  [-0.2848232 ]
  [ 0.46502182]
  [-0.5435326 ]
  [ 0.18193091]
  [ 0.46246475]
  [-0.675353  ]
  [ 0.14148118]
  [-0.51159894]
  [ 0.98774344]
  [-0.27734083]
  [ 0.5016268 ]
  [-0.18938245]
  [-0.75955355]
  [ 0.8700379 ]
  [-0.41848293]
  [-0.4404454 ]
  [-0.19864926]
  [-0.95759344]
  [-0.0616031 ]
  [ 0.3033899 ]
  [ 0.51998234]
  [-0.90784645]
  [ 0.35259104]
  [-0.41759104]
  [ 0.9059249 ]
  [-0.4141998 ]
  [-0.7611893 ]
  [-0.94

../output/GAN_1570439027/GAN_output_16
Composing from ../output/GAN_1570439027\GANmodel_weights_600.hdf5
[[[-0.997128  ]
  [-0.99841547]
  [-0.9909693 ]
  [-0.9984451 ]
  [-0.98723966]
  [-0.87888235]
  [-0.86412007]
  [-0.42767024]
  [-0.8271749 ]
  [-0.67616916]
  [-0.9625199 ]
  [-0.2515185 ]
  [ 0.9886222 ]
  [-0.94185525]
  [ 0.574375  ]
  [-0.8664497 ]
  [ 0.7815784 ]
  [ 0.0318366 ]
  [ 0.4118354 ]
  [-0.5661387 ]
  [ 0.5390955 ]
  [-0.5672984 ]
  [ 0.99622166]
  [-0.99421936]
  [ 0.9579597 ]
  [ 0.7425308 ]
  [ 0.96932274]
  [-0.8456208 ]
  [ 0.9126273 ]
  [ 0.339567  ]
  [ 0.91360897]
  [ 0.99301374]
  [-0.05854364]
  [-0.9872319 ]
  [-0.9933573 ]
  [-0.9440137 ]
  [-0.6395689 ]
  [ 0.7651999 ]
  [-0.04600135]
  [ 0.08682454]
  [-0.55084574]
  [ 0.43291503]
  [ 0.65934145]
  [ 0.6971731 ]
  [ 0.13200581]
  [ 0.9866702 ]
  [ 0.49140742]
  [ 0.21456607]
  [ 0.66956294]
  [ 0.4305897 ]
  [-0.9701034 ]
  [ 0.07403346]
  [ 0.59496933]
  [-0.51457286]
  [ 0.8677479 ]
  [ 0.5297437 ]

../output/GAN_1570439027/GAN_output_20


In [111]:
count = 0
for model_path in glob.glob(gan.outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    gan.generator.load_weights(model_path)
    prediction_notes = gan.generate_notes(possibleNotes)
    gan.create_midi(prediction_notes, gan.outputDest + 'GAN_output_' + str(count))
    print(gan.outputDest + 'GAN_output_' + str(count))
    count += 1

Composing from ../output/GAN_1570437586\GANmodel_weights_0.hdf5
[[[-0.79037344]
  [ 0.70460176]
  [-0.28947598]
  [ 0.8814572 ]
  [-0.6107855 ]
  [-0.38538986]
  [ 0.07783478]
  [ 0.41338134]
  [-0.8948215 ]
  [ 0.7541267 ]
  [ 0.42156288]
  [ 0.82168484]
  [-0.877071  ]
  [-0.3693905 ]
  [ 0.06191757]
  [-0.41833857]
  [ 0.6497538 ]
  [ 0.9913109 ]
  [ 0.9248787 ]
  [-0.99073195]
  [ 0.75921595]
  [-0.38791367]
  [ 0.7617763 ]
  [-0.08390729]
  [ 0.38214526]
  [-0.53922236]
  [ 0.99005866]
  [ 0.86944044]
  [ 0.02686271]
  [-0.9834003 ]
  [-0.44417801]
  [ 0.947354  ]
  [-0.4138857 ]
  [ 0.30625492]
  [ 0.8054296 ]
  [ 0.8645084 ]
  [-0.91852987]
  [-0.82418436]
  [-0.9886027 ]
  [ 0.2925736 ]
  [-0.9160985 ]
  [ 0.85405934]
  [ 0.70144534]
  [ 0.9964825 ]
  [ 0.19647111]
  [ 0.9931008 ]
  [-0.90167433]
  [ 0.8092278 ]
  [ 0.9887396 ]
  [ 0.9437407 ]
  [-0.97892994]
  [-0.27275044]
  [-0.42333934]
  [-0.60200846]
  [ 0.9684587 ]
  [-0.89766115]
  [ 0.79444194]
  [ 0.92340887]
  [-0.90

[[[-0.31863266]
  [-0.04901373]
  [ 0.9647185 ]
  [-0.9970886 ]
  [-0.7901506 ]
  [-0.5245192 ]
  [-0.99598765]
  [-0.98946106]
  [-0.97796994]
  [-0.7766503 ]
  [-0.9589379 ]
  [-0.90780485]
  [-0.89653635]
  [-0.83506596]
  [-0.9946219 ]
  [-0.698596  ]
  [ 0.70834357]
  [ 0.33375612]
  [-0.83358085]
  [-0.9176603 ]
  [-0.88028145]
  [-0.8599078 ]
  [ 0.9162866 ]
  [ 0.13114238]
  [-0.6726755 ]
  [-0.8211164 ]
  [-0.46654317]
  [ 0.22901817]
  [-0.96735495]
  [-0.28233814]
  [ 0.6000631 ]
  [ 0.6072935 ]
  [ 0.7140057 ]
  [ 0.82475835]
  [-0.14915876]
  [-0.96274424]
  [ 0.8794212 ]
  [-0.44175673]
  [ 0.39304227]
  [ 0.7064495 ]
  [-0.8201694 ]
  [ 0.12927328]
  [-0.85720026]
  [ 0.00342063]
  [ 0.10993685]
  [ 0.8549226 ]
  [-0.8549247 ]
  [-0.9479352 ]
  [-0.92161894]
  [-0.8428619 ]
  [ 0.01836698]
  [ 0.88301367]
  [ 0.4289415 ]
  [-0.65191066]
  [ 0.9797393 ]
  [ 0.300754  ]
  [-0.27318373]
  [-0.23616558]
  [ 0.92543495]
  [-0.91959906]
  [ 0.1948324 ]
  [ 0.7325499 ]
  [ 0.99

../output/GAN_1570437586/GAN_output_9
Composing from ../output/GAN_1570437586\GANmodel_weights_90.hdf5
[[[-0.99372005]
  [-0.07109336]
  [ 0.50417745]
  [ 0.6114273 ]
  [-0.99372274]
  [-0.93069106]
  [ 0.14586872]
  [-0.9409094 ]
  [-0.89703465]
  [-0.6764112 ]
  [ 0.91217065]
  [-0.08329976]
  [-0.6026658 ]
  [-0.9155475 ]
  [-0.4573485 ]
  [-0.2710771 ]
  [-0.83602333]
  [-0.9268878 ]
  [ 0.4406684 ]
  [ 0.5563792 ]
  [ 0.9190806 ]
  [-0.56058013]
  [ 0.567446  ]
  [ 0.8114215 ]
  [ 0.8143599 ]
  [-0.43429366]
  [-0.43725628]
  [ 0.1867322 ]
  [-0.52914596]
  [-0.9893395 ]
  [ 0.60228133]
  [-0.9499852 ]
  [-0.8755273 ]
  [ 0.9668063 ]
  [-0.04185651]
  [-0.33947974]
  [ 0.90217185]
  [ 0.85657346]
  [ 0.14158826]
  [-0.539097  ]
  [-0.33338523]
  [-0.90614957]
  [ 0.562485  ]
  [ 0.6537802 ]
  [ 0.3161599 ]
  [ 0.01375579]
  [-0.38288748]
  [ 0.56375265]
  [ 0.90417427]
  [ 0.99968565]
  [ 0.92991376]
  [ 0.87509125]
  [ 0.91637343]
  [ 0.95765615]
  [-0.5329507 ]
  [-0.8507352 ]
 

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [7]:
def generate_notes(model, notes, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    pitchnames = sorted(notes)
    
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
#         prediction_input = (prediction_input / float(n_vocab))*2 - 1

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]
    
    print(prediction_output)

    return prediction_output

# prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)

Next, I will create a midi using these notes and save to a file

In [8]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
# create_midi(prediction_output, outputDest + 'LSTM_output_final')

Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [ ]:
# Have each model make a song
count = 0
for model_path in glob.glob(outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1